In [1]:
from fourInRowGame import Chip, FourInRowGame
from randomAgent import randomFiarAgent
from actorCriticArena import newAgent, AversaryWrapperTfModel

/Users/adrian/Code/education/DMUU/four-in-a-row-rf-learning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
env = FourInRowGame(5, 6)
randomAgent = randomFiarAgent(env)
env.print()


- - - - - - 
- - - - - - 
- - - - - - 
- - - - - - 
- - - - - - 

In [63]:
env.reset()

In [4]:
yellowAgent = newAgent(env, 196, randomAgent, Chip.YELLOW)
adversary_1 = AversaryWrapperTfModel(yellowAgent)

100%|██████████| 10000/10000 [00:17<00:00, 579.51it/s, episode_reward=100, running_reward=-78] 


Solved at episode 9999: average reward: -78.00!


In [5]:
col = adversary_1.select_col(env)
env.play_game(Chip.YELLOW,col)
env.print()


- - - Y - - 
- - - Y - - 
- - - Y - - 
- R R Y R - 
- R Y Y R Y 

/Users/adrian/Code/education/DMUU/four-in-a-row-rf-learning/fourInRowGame.py:32: Warning: One player played multiple times in a row
  warnings.warn("One player played multiple times in a row", Warning)


In [6]:
env.play_game(Chip.RED,1)
env.print()


- - - Y - - 
- - - Y - - 
- R - Y - - 
- R R Y R - 
- R Y Y R Y 

In [23]:
redAgent = newAgent(env, 196, adversary_1, Chip.RED)
adversary_2 = AversaryWrapperTfModel(redAgent)

100%|██████████| 10000/10000 [00:42<00:00, 237.98it/s, episode_reward=100, running_reward=100] 


Solved at episode 9999: average reward: 100.00!


In [70]:
col = adversary_2.select_col(env)
env.play_game(Chip.RED,col)
env.print()


- - - - - - 
- R - - - - 
Y R - - - - 
Y R - - - - 
Y R - - - - 

In [71]:
col = adversary_1.select_col(env)
env.play_game(Chip.YELLOW,0)
env.print()


- - - - - - 
Y R - - - - 
Y R - - - - 
Y R - - - - 
Y R - - - - 